In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
#train_dataset = datasets.MNIST(root='/root/data/', train=True, transform=transforms.ToTensor(), download=True)
train_dataset = datasets.MNIST(root='/root/data/', train=True, transform=transforms.ToTensor(), download=False)
#test_dataset = datasets.MNIST(root='/root/data/', train=False, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='/root/data/', train=False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=True)

In [2]:
import torch
import torch.nn as nn


class VAE(nn.Module):

    def __init__(self, input_size, hidden_size, latent_dim) -> None:
        super().__init__()
        
        # Encoder
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc21 = nn.Linear(hidden_size, latent_dim)
        self.fc22 = nn.Linear(hidden_size, latent_dim)
        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_size)
        self.fc4 = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        #encoded = self.encoder(x)
        encoded = torch.flatten(x, 1)
        encoded = torch.tanh(self.fc1(encoded))
        mean = self.fc21(encoded)
        logvar = self.fc22(encoded)
        eps = torch.randn_like(logvar)
        std = torch.exp(logvar / 2)
        z = eps * std + mean
        x = torch.tanh(self.fc3(z))
        x = torch.sigmoid(self.fc4(x))
        x = torch.reshape(x, (-1, 1, 28, 28))
        decoded = x
        return mean, logvar, decoded


In [10]:
learning_rate = 2e-2
input_size = 784
hidden_size = 500
latent_size = 3
from time import time

n_epochs = 2000

def compute_elbo(x, reconst_x, mean, log_var):
    reconst_error = -torch.nn.functional.binary_cross_entropy(reconst_x, x, reduction='sum')
    kl_divergence = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    elbo = (reconst_error - kl_divergence) / len(x)
    return elbo

def train(device, model, latent_dim):
    print(latent_dim)
    #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)
    begin_time = time()
    # train
    loss_list = []
    with open('/root/paper/elbo/mnist_3.txt', 'w') as file:
        for i in range(n_epochs):
            loss_sum = 0
            for batch_idx, (x, _) in enumerate(train_loader):
                x = x.to(device)
                mean, logvar, x_hat = model(x)
                loss = -compute_elbo(x, x_hat, mean, logvar)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #loss_sum += loss.item()*len(x)
                #if batch_idx % 100 == 0:
                #    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(i, batch_idx * len(x), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))

            # estimate elbo
            model.eval()
            with torch.no_grad():
                each_epoch = 10
                n_samples = 100
                #train
                indices = torch.randperm(len(train_dataset))[:n_samples]
                x = torch.stack([train_dataset[i][0] for i in indices]).to(device)
                mean, logvar, x_hat = model(x)
                loss_sum = -compute_elbo(x, x_hat, mean, logvar)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.4f}'.format(i, loss_sum))
                file.write(str(-loss_sum.item()) + ' ')
                #test
                indices = torch.randperm(len(test_dataset))[:n_samples]
                x = torch.stack([test_dataset[i][0] for i in indices]).to(device)
                mean, logvar, x_hat = model(x)
                loss_sum = -compute_elbo(x, x_hat, mean, logvar)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.4f}'.format(i, loss_sum))
                file.write(str(-loss_sum.item()) + '\n')
            
            #time
            if(i % each_epoch == 0):
                training_time = time() - begin_time
                minute = int(training_time // 60)
                second = int(training_time % 60)
                print(f'time loss {minute}:{second}')
        
        torch.save(model, '/root/paper/elbo/mnist_3_model.pth')
        
    tot_training_time = time() - begin_time
    minute = int(tot_training_time // 60)
    second = int(tot_training_time % 60)
    print(f'total time loss {minute}:{second}')

def initialize_parameters(model):
    for param in model.parameters():
        param.data.normal_(mean=0, std=0.01)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.interpolate import interp1d
        
import numpy as np
import pandas as pd

def moving_average(data, window_size):
    return pd.Series(data).rolling(window=window_size, min_periods=1).mean().values

def drawing():
    plt.clf()
    fig = plt.figure(figsize=(3, 3))
    trains = []
    tests = []
    with open('/root/paper/elbo/mnist_3.txt', 'r') as file:
        for line in file:
            parts = line.split()
            trains.append(float(parts[0]))
            tests.append(float(parts[1]))
            
    epochs = [len(train_dataset) * i for i in range(1, len(trains) + 1)]
    
    trains_smoothed = moving_average(trains, 500)
    tests_smoothed = moving_average(tests, 500)

    plt.plot(epochs, tests_smoothed, label='Test')
    plt.plot(epochs, trains_smoothed, label='Train')
    
    plt.xlabel('Samples')
    plt.ylabel('ELBO')
    plt.title('MNIST, d=3')
    plt.legend()
    plt.ylim(-150, -100)
    plt.xlim(1e5, 1e8)
    plt.xscale('log')
    plt.savefig('/root/paper/elbo/pictures/mnist_3.png',bbox_inches='tight')
    plt.show()

In [ ]:
def main():
    device = 'cuda:0'

    #model = VAE(latent_dim = 10).to(device)

    # Load the model
    #model = torch.load('/root/VAE/VAE_model.pth', map_location=device)

    #Choose which to play
    model = VAE(input_size, hidden_size, latent_size).to(device)
    initialize_parameters(model)
    train(device, model, latent_size)
    #generate(device, model)
    #reconstruct(device, model)
    #latent_space(device, model)
    #explore_latent()
    #print(sum(p.numel() for p in model.parameters() if p.requires_grad))

if __name__ == '__main__':
    main()

In [ ]:
drawing()